In [ ]:
# --- CELL 1: SETUP & IMPORTS ---
# This cell installs and imports all required libraries.

!pip install ultralytics pandas scikit-learn

import cv2
import os
import numpy as np
import pandas as pd
from ultralytics import YOLO
from google.colab import drive, files
from sklearn.metrics import mean_absolute_error
import torch

# Set a seed for reproducibility
SEED_VALUE = 42
np.random.seed(SEED_VALUE)
torch.manual_seed(SEED_VALUE)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED_VALUE)

print("✅ All libraries imported.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.0 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ All libraries imported.


In [ ]:
# --- CELL 2: MOUNT DRIVE & LOAD MODELS ---
# This cell mounts Google Drive to load your trained models.

from google.colab import drive
print("Mounting Google Drive...")
drive.mount('/content/drive')
print("✅ Drive mounted.")

# --- 1. Load your trained "Tuned" YOLO model from Drive ---
MODEL_DRIVE_PATH = '/content/drive/My Drive/VisionAssist-Models/yolov8n_optuna_best.pt' # <-- Adjust this path if needed
if not os.path.exists(MODEL_DRIVE_PATH):
    print(f"--- ⛔ ERROR: 'best.pt' not found at {MODEL_DRIVE_PATH} ---")
    raise FileNotFoundError("Trained YOLO model not found in Google Drive.")
else:
    print(f"Found Tuned model in Google Drive: {MODEL_DRIVE_PATH}")

yolo_model_tuned = YOLO(MODEL_DRIVE_PATH)
print("✅ Tuned YOLO model loaded.")

# --- 2. Load the "Baseline" YOLO model ---
print("Loading Baseline YOLO (yolov8n.pt)...")
yolo_model_baseline = YOLO('yolov8n.pt') # The default, untuned model
print("✅ Baseline YOLO model loaded.")

print("✅ All models are ready.")

Mounting Google Drive...
Mounted at /content/drive
✅ Drive mounted.
Found Tuned model in Google Drive: /content/drive/My Drive/VisionAssist-Models/yolov8n_optuna_best.pt
✅ Tuned YOLO model loaded.
Loading Baseline YOLO (yolov8n.pt)...
✅ Baseline YOLO model loaded.
✅ All models are ready.


In [ ]:
# --- Define the path to your KITTI data ---
# (Adjust this if you used a different folder name)
KITTI_TRAINING_PATH = '/content/drive/My Drive/KITTI_data/training'

# --- Verify the paths ---
kitti_image_dir = os.path.join(KITTI_TRAINING_PATH, 'image_2')
kitti_label_dir = os.path.join(KITTI_TRAINING_PATH, 'label_2')

if not os.path.exists(kitti_image_dir) or not os.path.exists(kitti_label_dir):
    print("--- ⛔ ERROR: KITTI dataset not found at the expected location! ---")
    print(f"Please check your paths. We looked for:")
    print(f"- {kitti_image_dir}")
    print(f"- {kitti_label_dir}")
    print("\nPlease complete the manual download steps described above.")
else:
    print("✅ KITTI dataset found!")
    print(f"Images: {kitti_image_dir}")
    print(f"Labels: {kitti_label_dir}")

✅ KITTI dataset found!
Images: /content/drive/My Drive/KITTI_data/training/image_2
Labels: /content/drive/My Drive/KITTI_data/training/label_2


In [ ]:
# --- CELL 4: HELPER FUNCTIONS (PARSERS & HEURISTIC) ---

def estimate_distance(box_height_px, object_real_height_m, focal_length_px):
    """Your heuristic formula."""
    if box_height_px <= 0: return float('inf')
    return (object_real_height_m * focal_length_px) / box_height_px

def parse_kitti_label(label_path):
    """
    Parses a KITTI label file.
    Each line: type, truncated, occluded, alpha, bbox_left, bbox_top,
    bbox_right, bbox_bottom, dim_H, dim_W, dim_L, loc_X, loc_Y, loc_Z, rot_Y
    """
    gt_objects = []
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split(' ')
            obj_type = parts[0].lower() # e.g., 'Car', 'Pedestrian'

            # We only care about common classes your model knows
            # Map KITTI 'Car' to YOLO 'car', 'Pedestrian' to 'person', etc.
            if obj_type == 'car':
                yolo_class_name = 'car'
            elif obj_type == 'pedestrian':
                yolo_class_name = 'person'
            elif obj_type == 'cyclist':
                yolo_class_name = 'bicycle'
            else:
                continue # Skip objects we don't care about (e.g., 'Tram')

            gt_objects.append({
                'class_name': yolo_class_name,
                'bbox_2d': [float(p) for p in parts[4:8]], # [left, top, right, bottom]
                'real_height_m': float(parts[8]),         # dimensions H
                'true_distance_m': float(parts[13])       # location Z
            })
    return gt_objects

def calculate_iou(boxA, boxB):
    """Calculates Intersection over Union (IoU) between two boxes."""
    # box format: [x1, y1, x2, y2]
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

print("✅ Helper functions defined.")

✅ Helper functions defined.


In [ ]:
# --- CELL 5: THE MAIN CALIBRATION FUNCTION (v2 - More Logging) ---
# This version now logs box_height and real_height so we can calibrate.

def run_kitti_calibration(yolo_model, kitti_path, focal_length, conf_thresh, iou_threshold=0.5):

    print(f"\n--- Starting Calibration ---")
    print(f"  Model: {yolo_model.ckpt_path}")
    print(f"  Confidence: {conf_thresh}, Focal Length: {focal_length}px")

    image_dir = os.path.join(kitti_path, 'image_2')
    label_dir = os.path.join(kitti_path, 'label_2')

    # We will only test a sample (e.g., first 100 images)
    image_files = sorted(os.listdir(image_dir))[:100]

    yolo_class_names = yolo_model.names

    comparison_results = [] # Store (true_dist, pred_dist)

    for image_name in image_files:
        if not image_name.endswith('.png'):
            continue

        image_path = os.path.join(image_dir, image_name)
        label_path = os.path.join(label_dir, image_name.replace('.png', '.txt'))

        if not os.path.exists(label_path):
            continue

        # 1. Get Ground Truth (GT) objects from label file
        gt_objects = parse_kitti_label(label_path)

        # 2. Get Predicted objects from YOLO
        preds = yolo_model.predict(image_path, conf=conf_thresh, verbose=False)
        pred_boxes = preds[0].boxes.cpu().numpy()

        # 3. Match Predictions to Ground Truth using IoU
        for gt_obj in gt_objects:
            gt_bbox = gt_obj['bbox_2d']
            best_iou = 0
            best_match = None

            for i in range(len(pred_boxes)):
                pred_class_id = int(pred_boxes.cls[i])
                pred_class_name = yolo_class_names[pred_class_id]

                # Check if classes match
                if pred_class_name == gt_obj['class_name']:
                    pred_bbox = pred_boxes.xyxy[i]
                    iou = calculate_iou(gt_bbox, pred_bbox)

                    if iou > best_iou:
                        best_iou = iou
                        best_match = pred_bbox

            # 4. If we find a good match, log the distances
            if best_iou > iou_threshold:
                # Get the predicted box height
                y1, y2 = best_match[1], best_match[3]
                box_height_px = y2 - y1

                # Get GT info
                real_height_m = gt_obj['real_height_m']
                true_distance_m = gt_obj['true_distance_m']

                # Run your heuristic
                predicted_distance = estimate_distance(box_height_px, real_height_m, focal_length)

                # Log for MAE calculation
                if predicted_distance != float('inf') and box_height_px > 0:
                    comparison_results.append({
                        'true_distance': true_distance_m,
                        'predicted_distance': predicted_distance,
                        'class': gt_obj['class_name'],
                        'box_height_px': box_height_px,   # <-- NEW
                        'real_height_m': real_height_m    # <-- NEW
                    })

    print(f"--- Calibration Complete: Found {len(comparison_results)} matched objects. ---")
    return pd.DataFrame(comparison_results)

print("✅ Calibration function (v2) defined.")

✅ Calibration function (v2) defined.


In [ ]:
# --- CELL 6: RUN & ANALYZE RESULTS ---

# Define your heuristic's focal length
HEURISTIC_FOCAL_LENGTH = 1000

# --- 1. Run Baseline Model ---
baseline_results = run_kitti_calibration(
    yolo_model=yolo_model_baseline,
    kitti_path=KITTI_TRAINING_PATH,
    focal_length=HEURISTIC_FOCAL_LENGTH,
    conf_thresh=0.4 # Use baseline confidence
)

# --- 2. Run Tuned Model ---
tuned_results = run_kitti_calibration(
    yolo_model=yolo_model_tuned,
    kitti_path=KITTI_TRAINING_PATH,
    focal_length=HEURISTIC_FOCAL_LENGTH,
    conf_thresh=0.3 # Use your tuned confidence
)

# --- 3. Calculate MAE ---
print("\n" + "="*50)
print("--- FINAL MAE COMPARISON ---")
print(f" (Using {HEURISTIC_FOCAL_LENGTH}px focal length heuristic)")
print("="*50)

if not baseline_results.empty:
    baseline_mae = mean_absolute_error(baseline_results['true_distance'], baseline_results['predicted_distance'])
    print(f"  Baseline (yolov8n.pt) MAE: {baseline_mae:.2f} meters")
else:
    print("  Baseline (yolov8n.pt) MAE: N/A (No objects matched)")

if not tuned_results.empty:
    tuned_mae = mean_absolute_error(tuned_results['true_distance'], tuned_results['predicted_distance'])
    print(f"  Tuned (best.pt) MAE:     {tuned_mae:.2f} meters")
else:
    print("  Tuned (best.pt) MAE:     N/A (No objects matched)")

print("\n(Lower MAE is better)")

# --- Show a sample of the results ---
if not tuned_results.empty:
    print("\n--- Sample of Tuned Model's Results (in meters) ---")
    print(tuned_results.head())


--- Starting Calibration ---
  Model: yolov8n.pt
  Confidence: 0.4, Focal Length: 1000px
--- Calibration Complete: Found 237 matched objects. ---

--- Starting Calibration ---
  Model: /content/drive/My Drive/VisionAssist-Models/yolov8n_optuna_best.pt
  Confidence: 0.3, Focal Length: 1000px
--- Calibration Complete: Found 260 matched objects. ---

--- FINAL MAE COMPARISON ---
 (Using 1000px focal length heuristic)
  Baseline (yolov8n.pt) MAE: 7.31 meters
  Tuned (best.pt) MAE:     7.54 meters

(Lower MAE is better)

--- Sample of Tuned Model's Results (in meters) ---
   true_distance  predicted_distance   class  box_height_px  real_height_m
0           8.41           12.033261  person     157.064651           1.89
1          58.49           84.990540     car      19.649246           1.67
2          34.38           42.321938     car      33.316055           1.41
3          13.22           15.900142     car      98.741257           1.57
4          38.26           53.121014     car      

In [ ]:
# --- CELL 7: CALIBRATE FOCAL LENGTH ---
!pip install scipy -q
from scipy.optimize import minimize

def find_best_focal_length(results_df):
    """
    Finds the optimal focal length to minimize MAE.
    """
    if results_df.empty:
        return None, float('inf')

    # Define the function to minimize
    # 'f' is the focal_length we are trying to find
    def calculate_mae_for_focal_length(f):
        # We must use .values to avoid pandas index issues
        # f[0] is because 'minimize' passes 'f' as an array
        predicted_distances = (results_df['real_height_m'].values * f[0]) / results_df['box_height_px'].values

        # Calculate MAE between our new predictions and the truth
        mae = np.mean(np.abs(predicted_distances - results_df['true_distance'].values))
        return mae

    # Start with an initial guess (1000)
    initial_guess = [1000.0]

    # Run the optimization
    result = minimize(calculate_mae_for_focal_length, initial_guess, method='Nelder-Mead')

    if result.success:
        best_focal_length = result.x[0]
        min_mae = result.fun
        return best_focal_length, min_mae
    else:
        return None, float('inf')

print("\n" + "="*50)
print("--- CALIBRATING FOCAL LENGTH ---")
print("="*50)

# Calibrate using the data from the Tuned model (which has more detections)
best_f, tuned_mae_calibrated = find_best_focal_length(tuned_results)

if best_f:
    print(f"✅ Calibration Successful!")
    print(f"  Your old guess was 1000px, which gave an MAE of {tuned_mae:.2f}m")
    print(f"  The OPTIMAL focal length is: {best_f:.2f}px")
    print(f"  This gives a NEW, CALIBRATED MAE of: {tuned_mae_calibrated:.2f}m")

    # Now, let's see what the Baseline MAE is using this *same* calibrated focal length
    baseline_preds_calibrated = (baseline_results['real_height_m'] * best_f) / baseline_results['box_height_px']
    baseline_mae_calibrated = mean_absolute_error(baseline_results['true_distance'], baseline_preds_calibrated)

    print("\n" + "="*50)
    print("--- FINAL CALIBRATED RESULTS ---")
    print(f"  Calibrated Baseline MAE: {baseline_mae_calibrated:.2f}m")
    print(f"  Calibrated Tuned MAE:    {tuned_mae_calibrated:.2f}m")
    print("="*50)

    print("\nThis is the comparison you should put in your report.")

else:
    print("--- ⛔ Calibration failed. ---")


--- CALIBRATING FOCAL LENGTH ---
✅ Calibration Successful!
  Your old guess was 1000px, which gave an MAE of 7.54m
  The OPTIMAL focal length is: 762.99px
  This gives a NEW, CALIBRATED MAE of: 2.38m

--- FINAL CALIBRATED RESULTS ---
  Calibrated Baseline MAE: 2.18m
  Calibrated Tuned MAE:    2.38m

This is the comparison you should put in your report.
